this file is generated by freecodecamp algo tranding session
https://github.com/nickmccullum/algorithmic-trading-python/blob/master

In [ ]:
pip install pandas
pip install numpy
pip install xlsxwriter

In [1]:
import numpy as np
import pandas as pd
import math
import requests
import xlsxwriter

read list of 500 stocks

In [21]:
stocks=pd.read_csv('sp_500_stocks.csv')

aquire API token

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

making first call

In [9]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
api_url

'https://sandbox.iexapis.com/stable/stock/AAPL/quote/?token=Tpk_059b97af715d417d9f49f50b51b1c448'

In [ ]:
data=requests.get(api_url).json()
data

In [13]:
price=data['latestPrice']
marketcap=data['marketCap']/1000000000000


2.312874104675

In [14]:
my_columns=['ticker','stockprice','marketcap','numberofsharetobuy']
final_dataframe=pd.DataFrame(columns=my_columns)

In [17]:
final_dataframe.append(
    pd.Series(
        [symbol,price,marketcap,'N/A'] , index=my_columns
    ), ignore_index=True
)

,ticker,stockprice,marketcap,numberofsharetobuy
0,AAPL,145.11,2.312874,N/A


In [22]:
final_dataframe=pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:2]:
    api_url=f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series(
        [stock,data['latestPrice'],data['marketCap'],'N/A'] , index=my_columns
    ), ignore_index=True
    )

In [23]:
final_dataframe

,ticker,stockprice,marketcap,numberofsharetobuy
0,A,129.36,38686628710,N/A
1,AAL,12.62,8302898463,N/A


using batch api mode

In [24]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [ ]:
symbolgroups=list(chunks(stocks['Ticker'],100))
symbolstrings=[]
for i in range(0,len(symbolgroups)):
    symbolstrings.append(','.join(symbolgroups[i]))

final_dataframe=pd.DataFrame(columns=my_columns)
for symbolstrings in symbolstrings:
    batch_api_url=f''

#yt run time 1:04